In [20]:
cd desktop

/Users/christopherbaker_1/Desktop


In [21]:
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_key import geoapify_key


In [37]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city

In [95]:
# Configure the map plot
coordinates = {
    "Lat": (city_data_df["Lat"]),
    "Lng": (city_data_df["Lng"]),
    "City": (city_data_df["City"]),
    "Humidity": (city_data_df["Humidity"])
}

coordinates_df = pd.DataFrame(coordinates)

humidity_map = coordinates_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM",
                                            frame_width = 700, frame_height = 500,
                                            size = "Humidity", color = "City"
)

humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [60]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df["Humidity"] < 40)
        & (city_data_df["Max Temp"] > 20)
        & (city_data_df["Cloudiness"] > 25)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,rio gallegos,-51.6226,-69.2181,20.03,26,75,8.23,AR,1666108296
103,103,siavonga,-16.5382,28.7088,31.47,32,71,1.58,ZM,1666108296
105,105,nova olimpia,-14.7972,-57.2881,40.11,21,93,1.65,BR,1666108147
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219
162,162,sao miguel do araguaia,-13.2750,-50.1628,40.09,13,33,3.87,BR,1666108241


### Step 3: Create a new DataFrame called `hotel_df`.

In [62]:
# Use Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Humidity", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name", " ")

# Display sample data
hotel_df.head()

,City,Country,Humidity,Lat,Lng,Hotel Name
102,rio gallegos,AR,26,-51.6226,-69.2181,
103,siavonga,ZM,32,-16.5382,28.7088,
105,nova olimpia,BR,21,-14.7972,-57.2881,
106,palencia,ES,30,42.4167,-4.5000,
162,sao miguel do araguaia,BR,13,-13.2750,-50.1628,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [74]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":15
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
siavonga - nearest hotel: Lake Safari Lodge Annex
nova olimpia - nearest hotel: Olímpia Hotel
palencia - nearest hotel: No hotel found
sao miguel do araguaia - nearest hotel: Hotel São Miguel
ouallam - nearest hotel: No hotel found
shakawe - nearest hotel: Drotsky's Cabins
gwanda - nearest hotel: No hotel found
cabra - nearest hotel: Pensión Guerrero
prieska - nearest hotel: No hotel found
sakakah - nearest hotel: فندق النزل
viseu - nearest hotel: No hotel found
beitbridge - nearest hotel: Beitbridge Inn & Casino
adrar - nearest hotel: No hotel found
buraydah - nearest hotel: دنا للوحدات السكنية
bogande - nearest hotel: No hotel found
tessalit - nearest hotel: No hotel found
barroso - nearest hotel: Hotel Gold
araouane - nearest hotel: No hotel found
caiaponia - nearest hotel: No hotel found
nsanje - nearest hotel: No hotel found
nouadhibou - nearest hotel: Hotel valencia
kalabo - nearest hotel: Gol

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [88]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", 
                                        frame_width=600, frame_height=400, 
                                        size="Humidity", color="City", 
                                        hover_cols = ["Hotel Name","Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)